<a href="https://colab.research.google.com/github/suridianp/The-Classification-of-Coal-Mine-Pillar-Stability-Using-Stacking-Ensemble-Learning-Model/blob/main/Classification_Stacking_Ensemble_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===== INSTALLING =====
# !pip install xgboost
!pip install imbalanced-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# ===== DEPENDECIES =====
# General
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Google Drive
from google.colab import drive
# Tools
from imblearn.over_sampling import SMOTE
  # from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn import metrics
# from sklearn.metrics import plot_confusion_matrix # Deprecated
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
# Classifiers
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb

In [ ]:
# ===== DATASET =====
# Get data from Google Drive
drive.mount('/content/gdrive', force_remount=True)
# Set file to DataFrame
df = pd.read_csv("/content/gdrive/MyDrive/Pillar Stability/Data/new_clustered_data.csv")

In [ ]:
# Checking null/NaN value in dataset
df.isna().sum()
# df = df.dropna() # If any, run this code

Depth      0
BW         0
PW         0
MH         0
Cluster    0
dtype: int64

In [ ]:
# ===== ENCODING =====
# Changes "CLuster"s value
df["Cluster"].replace("F0", 0, inplace=True)
df["Cluster"].replace("F1", 1, inplace=True)
df["Cluster"].replace("I0", 2, inplace=True)
df["Cluster"].replace("I1", 3, inplace=True)
# Changes "Label"s data type
df["Cluster"].astype("float64")

In [ ]:
# ===== DEPENDENT AND INDEPENDENT FEATURE =====
target_variable = "Cluster"
predictors = ["Depth", "PW", "BW", "MH"]
X = df[predictors].values
y = df[target_variable].values

#####Ref: https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/

In [ ]:
# ===== OVERSAMPLING WITH SMOTE =====
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
# Define pipeline
over = SMOTE()
under = RandomUnderSampler()
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)
# Transform the dataset
X_resampled, y_resampled = pipeline.fit_resample(X, y)
# Create dataframe from resampled
X_resampled = pd.DataFrame(X_resampled)
y_resampled = pd.DataFrame(y_resampled, columns=['Cluster'])
df = pd.concat([X_resampled, y_resampled], axis=1)
# Rename Dataframe's Column
df.rename(columns={0: 'Depth', 1: 'BW', 2: "PW", 3: "MH"}, inplace=True)
# Printing total data from each columns
print(df.groupby(['Cluster']).agg({"Cluster": "count"}), end="\n\n\n") # Show data

##All Methods

In [ ]:
# ===== SPLITING DATASET =====
# Split dataset to training and testing dataset
def splitDataset(train, test):
  return train_test_split(X_resampled, y_resampled, train_size=train, test_size=test)

In [ ]:
# ===== HYPERPARAMETER TUNING =====
def setHyperparameter(X_train, y_train):
  random_grid_rf = {'n_estimators': [20,50,100,150], # Number of trees in the random forest
                'max_depth': [int(x) for x in np.linspace(10, 120, num = 12)], # Maximum number of levels allowed in each decision tree
                'min_samples_split': [6, 10, 15], # Minimum sample number to split a node
                'min_samples_leaf': [3, 4, 6], # Minimum sample number that can be stored in a leaf node
                'bootstrap': [True, False]} # Method used to sample data points
  random_grid_xgb = {'max_depth': [2, 3, 6, 10],
                'learning_rate': [0.01, 0.1, 0.2, 0.3],
                'subsample': np.arange(0.1, 1.0, 0.1),
                'colsample_bytree': np.arange(0.5, 1.0, 0.1),
                'colsample_bylevel': np.arange(0.5, 1.0, 0.1),
                'n_estimators': [50, 100, 250, 500],
                'num_class': [5, 10, 15]
                }
  random_grid_gbdt = {"n_estimators":[20,50,100, 150],
                "max_depth":[1,3,5,7,9],
                "learning_rate":[0.01,0.1,1,10,100]}
  # Randomized search on hyper parameters.
  rf_random = RandomizedSearchCV(estimator = RandomForestClassifier(),
                                param_distributions = random_grid_rf,
                                n_iter = 100,
                                cv = 5,
                                verbose = 2,
                                random_state = 35,
                                n_jobs = -1)
  xgb_random = RandomizedSearchCV(estimator = xgb.XGBClassifier(),
                            param_distributions = random_grid_xgb,
                            scoring = 'accuracy',
                            n_iter = 25,
                            n_jobs = 4,
                            verbose = 1,
                            error_score = 'raise')
  gbdt_random = RandomizedSearchCV(estimator = GradientBoostingClassifier(),
                                  param_distributions = random_grid_gbdt,
                                  n_iter = 100,
                                  cv = 5,
                                  verbose = 2,
                                  random_state = 35,
                                  n_jobs = -1)
  # Fit randomized search result to train and test dataset
  rf_random = rf_random.fit(X_train, y_train)
  xgb_random = xgb_random.fit(X_train, y_train)
  gbdt_random = gbdt_random.fit(X_train, y_train)
  return rf_random, xgb_random, gbdt_random

In [ ]:
# ===== DEFAULT CLASSIFIER'S PARAMETERS =====
ParamsRandomForest = {
  'n_estimators':50, 'max_depth':60, 'min_samples_split':10, 'min_samples_leaf':3, 'bootstrap':True
}
ParamsXGBoost = {
  'subsample':1, 'num_class':15, 'n_estimators':100, 'max_depth':6, 'learning_rate':0.3, 'colsample_bytree':1, 'colsample_bylevel':1
  # num_class dan n_estimator not set for default because no references
}
ParamsGBDT = {
  'n_estimators':100, 'max_depth':3, 'learning_rate':0.1
}

In [ ]:
# ===== CHANGE BEST PARAMETERS =====
def changeBestParams(prf, pxgb, pgbdt, rfr, xgbr, gbdtr):
  for x in rfr.items():
    for y in prf.items():
      if x[0] == y[0]:
        prf[y[0]] = x[1]
  for x in xgbr.items():
    for y in pxgb.items():
      if x[0] == y[0]:
        pxgb[y[0]] = x[1]
  for x in gbdtr.items():
    for y in pgbdt.items():
      if x[0] == y[0]:
        pgbdt[y[0]] = x[1]
  return prf, pxgb, pgbdt

In [ ]:
# ===== SET BEST PARAMETERS =====
def setBestParams(prf, pxgb, pgbdt):
    rfc_hp = RandomForestClassifier(
      n_estimators = prf['n_estimators'],
      max_depth = prf['max_depth'],
      min_samples_split = prf['min_samples_split'],
      min_samples_leaf = prf['min_samples_leaf'],
      bootstrap = prf['bootstrap']
    )
    xgb_hp = xgb.XGBClassifier(
      subsample = pxgb['subsample'],
      num_class = pxgb['num_class'],
      n_estimators = pxgb['n_estimators'],
      max_depth = pxgb['max_depth'],
      learning_rate = pxgb['learning_rate'],
      colsample_bytree = pxgb['colsample_bytree'],
      colsample_bylevel = pxgb['colsample_bylevel']
    )
    gbdt_hp = GradientBoostingClassifier(
      n_estimators = pgbdt['n_estimators'],
      max_depth = pgbdt['max_depth'],
      learning_rate = pgbdt['learning_rate']
    )
    return rfc_hp, xgb_hp, gbdt_hp

In [ ]:
# ===== SET CLASSIFIER =====
def setClassifier(rfc_hp, xgb_hp, gbdt_hp):
  layer_one_estimators = [
      ('rf', rfc_hp),
      ('xgb', xgb_hp),
      ('gbdt', gbdt_hp)]
  base_classifier_hyperparameter = [
      ('rf', rfc_hp),
      ('xgb', xgb_hp),
      ('gbdt', gbdt_hp)
  ]
  # last_layer = StackingClassifier(estimators=base_classifier_hyperparameter, final_estimator=LogisticRegression())
  last_layer = StackingClassifier(estimators=base_classifier_hyperparameter, final_estimator=rfc_hp)
  return layer_one_estimators, last_layer

In [ ]:
# ===== EVALUATION PROCESS =====
def evaluationProcess(evaluation_information, name, model, X_train, X_test, y_train, y_test):
  start_time = time.time()
  model.fit(X_train, y_train)
  prediction = model.predict(X_test)
  end_time = time.time()
  mod_t = end_time - start_time
  evaluation_information['ct'].append(mod_t)
  evaluation_information['name'].append(name)
  acc = metrics.accuracy_score(y_test, prediction)
  evaluation_information['acc'].append(acc)
  precision = metrics.precision_score(y_test, prediction, pos_label=1, average='weighted')  # calculate precision
  evaluation_information['prec'].append(precision)
  recall_sensitivity = metrics.recall_score(y_test, prediction, pos_label=1, average='weighted')  # calculate recall sensitivity
  evaluation_information['recall'].append(recall_sensitivity)
  f1 = metrics.f1_score(y_test, prediction, pos_label=1, average='weighted')  # calculate f1 score
  evaluation_information['f1'].append(f1)
  return evaluation_information

In [ ]:
# ===== GET EVALUATION RESULT =====
def getEvaluationResult(layer_one_estimators, last_layer, X_train, X_test, y_train, y_test):
  evaluation_information = {
      'name': [],
      'acc': [],
      'prec': [],
      'recall': [],
      'f1': [],
      'ct': []
  }
  for name, model in layer_one_estimators:
    evaluation_information = evaluationProcess(evaluation_information, name, model, X_train, X_test, y_train, y_test)
  evaluation_information = evaluationProcess(evaluation_information, 'stacked', last_layer, X_train, X_test, y_train, y_test)
  return evaluation_information

#Changes Something in Here!

In [ ]:
# ===== SET SPLITTING PERCENTAGES =====
train_spliting_decision = [0.65, 0.7, 0.75, 0.8, 0.85]
# train_spliting_decision = [0.75]
test_spliting_decision = [0.35, 0.3, 0.25, 0.2, 0.15]
# test_spliting_decision = [0.25]
allEvalResult = []
looping_number = 10
# looping_number = 1

In [ ]:
# ===== MAIN PROCESS =====
for x in range(0, len(train_spliting_decision)):
  X_train, X_test, y_train, y_test = splitDataset(train_spliting_decision[x], test_spliting_decision[x])
  rf_random, xgb_random, gbdt_random = setHyperparameter(X_train, y_train)
  # SET MODEL
  prf, pxgb, pgbdt = changeBestParams(
      ParamsRandomForest,
      ParamsXGBoost,
      ParamsGBDT,
      rf_random.best_params_,
      xgb_random.best_params_,
      gbdt_random.best_params_,
  )
  rfc_hp, xgb_hp, gbdt_hp = setBestParams(prf, pxgb, pgbdt)
  layer_one_estimators, last_layer = setClassifier(rfc_hp, xgb_hp, gbdt_hp)
  splitEvalResultInstance = []
  for x in range(0, looping_number):
    evaluation_information = getEvaluationResult(layer_one_estimators, last_layer, X_train, X_test, y_train, y_test)

    splitEvalResultInstance.append(evaluation_information)
  allEvalResult.append(splitEvalResultInstance)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


Fitting 5 folds for each of 25 candidates, totalling 125 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
<ipython-input-15-2137538f9481>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


...

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
<ipython-input-15-2137538f9481>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


Fitting 5 folds for each of 25 candidates, totalling 125 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
<ipython-input-15-2137538f9481>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)

In [ ]:
# ===== SHOW TABLE RESULT (SPLITTING) PER-LOOPING =====
for x in range(0, len(train_spliting_decision)):
  train_perc = train_spliting_decision[x]
  test_perc = 1 - train_perc
  print('Untuk dataset dengan splitting "%.2f":"%.2f" (Train:Test)' % (train_perc, test_perc))
  for y in range(0,looping_number):
    print('Pengulangan ke %s' % (y))
    datF = pd.DataFrame(allEvalResult[x][y])
    display(datF)
  print('|')
  print('|')

Untuk dataset dengan splitting "0.65":"0.35" (Train:Test)
Pengulangan ke 0


,name,acc,prec,recall,f1,ct
0,rf,0.913649,0.914756,0.913649,0.913573,0.050957
1,xgb,0.913649,0.913426,0.913649,0.913409,1.616969
2,gbdt,0.922006,0.922822,0.922006,0.921933,0.327708
3,stacked,0.905292,0.904950,0.905292,0.905051,3.051929


Pengulangan ke 1


,name,acc,prec,recall,f1,ct
0,rf,0.905292,0.906422,0.905292,0.905177,0.035421
1,xgb,0.913649,0.913426,0.913649,0.913409,0.235181
2,gbdt,0.919220,0.921733,0.919220,0.918946,0.325148
3,stacked,0.913649,0.914240,0.913649,0.913595,3.129811


Pengulangan ke 2


...

Pengulangan ke 8


,name,acc,prec,recall,f1,ct
0,rf,0.941558,0.947100,0.941558,0.942035,0.092124
1,xgb,0.948052,0.954870,0.948052,0.948675,0.433378
2,gbdt,0.928571,0.938111,0.928571,0.929291,1.409329
3,stacked,0.954545,0.963384,0.954545,0.955121,11.058973


Pengulangan ke 9


,name,acc,prec,recall,f1,ct
0,rf,0.941558,0.947100,0.941558,0.942035,0.096222
1,xgb,0.948052,0.954870,0.948052,0.948675,0.410998
2,gbdt,0.935065,0.946901,0.935065,0.935719,1.444885
3,stacked,0.948052,0.959284,0.948052,0.948727,11.132607


|
|


In [ ]:
# ===== GET ACCURACY PER-MODEL =====
mean_acc_random_forest = []
mean_acc_xgboost = []
mean_acc_gbdt = []
mean_acc_stacked = []
mean_time_random_forest = []
mean_time_xgboost = []
mean_time_gbdt = []
mean_time_stacked = []

for x in range(0, len(train_spliting_decision)):
  mean_rf = 0
  mean_xgb = 0
  mean_gbdt = 0
  mean_s = 0
  mean_t_rf = 0
  mean_t_xgb = 0
  mean_t_gbdt = 0
  mean_t_s = 0
  for y in range(0, looping_number):
    mean_rf += allEvalResult[x][y]['acc'][0]
    mean_xgb += allEvalResult[x][y]['acc'][1]
    mean_gbdt += allEvalResult[x][y]['acc'][2]
    mean_s += allEvalResult[x][y]['acc'][3]
    mean_t_rf += allEvalResult[x][y]['ct'][0]
    mean_t_xgb += allEvalResult[x][y]['ct'][1]
    mean_t_gbdt += allEvalResult[x][y]['ct'][2]
    mean_t_s += allEvalResult[x][y]['ct'][3]
  mean_rf = mean_rf/looping_number
  mean_xgb = mean_xgb/looping_number
  mean_gbdt = mean_gbdt/looping_number
  mean_s = mean_s/looping_number
  mean_t_rf = mean_t_rf/looping_number
  mean_t_xgb = mean_t_xgb/looping_number
  mean_t_gbdt = mean_t_gbdt/looping_number
  mean_t_s = mean_t_s/looping_number
  # Add to
  mean_acc_random_forest.append(mean_rf)
  mean_acc_xgboost.append(mean_xgb)
  mean_acc_gbdt.append(mean_gbdt)
  mean_acc_stacked.append(mean_s)
  mean_time_random_forest.append(mean_t_rf)
  mean_time_xgboost.append(mean_t_xgb)
  mean_time_gbdt.append(mean_t_gbdt)
  mean_time_stacked.append(mean_t_s)

In [ ]:
# ===== SHOW ACCURACY MODEL IN TABLE =====
to_data_frame = [
  mean_acc_random_forest,
  mean_acc_xgboost,
  mean_acc_gbdt,
  mean_acc_stacked,
]
last_data_frame = pd.DataFrame(to_data_frame)
last_data_frame['Model'] = ['RF', 'XGBoost', 'GBDT', 'Stacked']
last_data_frame.rename(columns={0: '65:35', 1: '70:30', 2: "75:25", 3: "80:20", 4:"85:15"}, inplace=True)
print("AKURASI RATA-RATA SETIAP MODEL DENGAN " + str(looping_number) + " KALI PENGULANGAN")
last_data_frame

AKURASI RATA-RATA SETIAP MODEL DENGAN 10 KALI PENGULANGAN


,65:35,70:30,75:25,80:20,85:15,Model
0,0.911699,0.920455,0.923438,0.925366,0.932468,RF
1,0.913649,0.931818,0.929688,0.941463,0.948052,XGBoost
2,0.911142,0.908117,0.932031,0.951220,0.937013,GBDT
3,0.910864,0.925000,0.931641,0.959024,0.947403,Stacked


In [ ]:
# ===== SHOW TIME COMPUTATION MODEL IN TABLE =====
time_to_data_frame = [
  mean_time_random_forest,
  mean_time_xgboost,
  mean_time_gbdt,
  mean_time_stacked,
]
last_time_data_frame = pd.DataFrame(time_to_data_frame)
last_time_data_frame['Model'] = ['RF', 'XGBoost', 'GBDT', 'Stacked']
last_time_data_frame.rename(columns={0: '65:35', 1: '70:30', 2: "75:25", 3: "80:20", 4:"85:15"}, inplace=True)
print("WAKTU KOMPUTASI RATA-RATA SETIAP MODEL DENGAN " + str(looping_number) + " KALI PENGULANGAN")
last_time_data_frame

WAKTU KOMPUTASI RATA-RATA SETIAP MODEL DENGAN 10 KALI PENGULANGAN


,65:35,70:30,75:25,80:20,85:15,Model
0,0.039464,0.090210,0.094544,0.042943,0.099463,RF
1,0.557335,1.129945,0.412109,0.656681,0.575915,XGBoost
2,0.308395,0.856167,0.379829,1.362792,1.431334,GBDT
3,3.285579,8.159392,3.299797,10.469234,11.332206,Stacked


In [ ]:
AER = pd.DataFrame(allEvalResult)
AER.to_csv("eval_res_SMOTEimproved_hparams_5split_10loop_MetaClassifier=RF(HP).csv", index=False)